In [1]:
import pandas as pd
import numpy as np
import os
import matplotlib.pyplot as plt
raw_data_path=os.path.join('data','raw')
df=pd.read_csv(os.path.join(raw_data_path,'fer2013.csv'))
X=[]
for i in range(len(df)):
    X.append(df.loc[i,'pixels'].split(' '))
X=np.array(X,np.float32)/255.
y=df.emotion.values
X_train=X[np.where(df.Usage=='Training')]
y_train=y[np.where((df.Usage=='Training'))]
X_test=X[np.where((df.Usage!='Training'))]
y_test=y[np.where((df.Usage!='Training'))]
def balance_classes(X,y):
    X1=X[np.where(y!=1)]
    X2=X[np.where(y==1)]
    X2=np.repeat(X2,9,axis=0)
    X=np.concatenate((X1,X2),axis=0)
    y1=y[np.where(y!=1)]
    y2=y[np.where(y==1)]
    y2=np.repeat(y2,9,axis=0)
    y=np.concatenate((y1,y2),axis=0)
    i=[i for i in range(len(y))]
    np.random.shuffle(i)
    return X[i],y[i]
X_train,y_train=balance_classes(X_train,y_train)
def one_hot_encoder(X):
    N=len(X)
    K=len(set(X))
    Z=np.zeros((N,K))
    for i in range(N):
        c=X[i]
        Z[i,c]=1
    return Z
y_train=one_hot_encoder(y_train)

In [ ]:
def forward(X,W,b):
    return softmax(X.dot(W)+b)
def softmax(a):
    return np.exp(a)/(np.exp(a).sum(axis=1,keepdims=True))
def cost(y,y_p):
    return -(y*np.log(y_p)).mean()
def fit(X,y,lr,epoch,fig=True):
    N,D=X.shape
    K=y.shape[1]
    W=np.random.randn(D,K)
    b=np.random.randn(K)
    tr_i=np.random.choice(N,int(N*0.9),replace=False)
    t_i=[i for i in range(N) if i not in tr_i]
    X_t=X[tr_i]
    y_t=y[tr_i]
    X_v=X[t_i]
    y_v=y[t_i]
    c_t=[]
    c_v=[]
    cl_t=[]
    cl_v=[]
    for i in range(epoch):
        y_p_t=forward(X_t,W,b)
        y_p_v=forward(X_v,W,b)
        dy=y_p_t-y_t
        W=W-lr*(X_t.T.dot(dy))
        b=b-lr*(dy).sum(axis=0)
        c_t.append(cost(y_t,y_p_t))
        c_v.append(cost(y_v,y_p_v))
        cl_t.append(score(y_t.argmax(axis=1),y_p_t.argmax(axis=1)))
        cl_v.append(score(y_v.argmax(axis=1),y_p_v.argmax(axis=1)))
        if i % 100==0:
            print('Epoch {0} Train Cost: {1} , Classification: {2} , Test Cost {3} , Classification : {4}'.format(i,c_t[i],
                                                                                                                  cl_t[i],
                                                                                                                  c_v[i],
                                                                                                                  cl_v[i]))
    if fig==True:
        plt.plot(c_t,label='Train Cost')
        plt.plot(c_v,label='Test Cost')
        plt.show()
        plt.plot(cl_t,label='Train Class')
        plt.plot(cl_v,label='Test Class')
        plt.show()
def score(y,y_p):
    return np.mean(y==y_p)